# Named Entity Recognition with BiLSTM


In [5]:
!pip install transformers datasets

In [6]:
# Import necessary libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

In [7]:
# Load the WikiANN dataset
dataset = load_dataset("wikiann", "ur")

In [8]:
# Access train, validation, and test datasets
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

# Replace 'ORG' tag with 'ORGANIZATION'
ner_data = dataset.map(lambda example: {
    'tokens': example['tokens'],
    'ner_tags': [tag.replace('ORG', 'ORGANIZATION') if isinstance(tag, str) else tag for tag in example['ner_tags']]
})

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [9]:
# Extract 'tokens' and 'ner_tags'
train_tokens = train_data['tokens']
train_ner_tags = train_data['ner_tags']

In [10]:
# Create word-to-index and tag-to-index mappings
word2idx = {}
tag2idx = {}
word_idx = 1
tag_idx = 1

for tokens, ner_tags in zip(train_tokens, train_ner_tags):
    for token, tag in zip(tokens, ner_tags):
        if token not in word2idx:
            word2idx[token] = word_idx
            word_idx += 1
        if tag not in tag2idx:
            tag2idx[tag] = tag_idx
            tag_idx += 1


In [11]:
# Define the BiLSTM model
class BiLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        predictions = self.fc(output)
        return predictions

In [12]:
# Convert tokens and tags to numerical values
train_data_numerical = [
    ([word2idx[token] for token in tokens], [tag2idx[tag] for tag in ner_tags])
    for tokens, ner_tags in zip(train_tokens, train_ner_tags)
]


In [13]:
# Hyperparameters
input_dim = len(word2idx) + 1
output_dim = len(tag2idx) + 1
embedding_dim = 100
hidden_dim = 256

In [14]:
# Initialize the model
model = BiLSTM(input_dim, embedding_dim, hidden_dim, output_dim)

In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    for tokens, tags in train_data_numerical:
        optimizer.zero_grad()
        tokens_tensor = torch.LongTensor(tokens)
        tags_tensor = torch.LongTensor(tags)
        predictions = model(tokens_tensor.unsqueeze(0))
        predictions = predictions.view(-1, predictions.shape[-1])
        tags_tensor = tags_tensor.view(-1)
        loss = criterion(predictions, tags_tensor)
        loss.backward()
        optimizer.step()



In [ ]:
# Convert numerical predictions back to tags for evaluation
def numerical_to_tags(predictions, idx2tag):
    return [idx2tag[pred.item()] for pred in predictions]

# Create reverse mappings for tag-to-index
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

In [ ]:
# evaluate the model
def evaluate_model(data):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for tokens, ner_tags in zip(data['tokens'], data['ner_tags']):
            tokens_tensor = torch.LongTensor([word2idx[token] for token in tokens])
            tags_tensor = torch.LongTensor([tag2idx[tag] for tag in ner_tags])
            predictions = model(tokens_tensor.unsqueeze(0)).argmax(dim=-1).squeeze().tolist()

            # Convert numerical predictions back to tags
            pred_tags = numerical_to_tags(predictions, idx2tag)
            all_predictions.extend(pred_tags)
            all_labels.extend(ner_tags)

    return all_predictions, all_labels


In [ ]:
# Evaluate on validation and test datasets
valid_predictions, valid_labels = evaluate_model(valid_data)
test_predictions, test_labels = evaluate_model(test_data)

In [ ]:
# Perform evaluation metrics calculations
from sklearn.metrics import precision_recall_fscore_support

# Calculate metrics for validation data
valid_precision, valid_recall, valid_f1, _ = precision_recall_fscore_support(valid_labels, valid_predictions, average='weighted')

# Calculate metrics for test data
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='weighted')

print("Validation Precision:", valid_precision)
print("Validation Recall:", valid_recall)
print("Validation F1 Score:", valid_f1)
print("\nTest Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)